In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import requests
import pprint
import time
from citipy import citipy

from config import weather_api_key

In [2]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [3]:
# 3. Create a new set of 2,000 random latitudes and longitudes
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [5]:
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

-72.959258279319 102.79193486583068
26.632684260374077 -155.29459839247642
11.267209960502939 60.00518220002101
2.807798055368096 -159.67221258242853
-14.426334514973348 -75.23808328462282
-33.06509120682598 -137.6012228271797
-59.076601519162324 118.12564948385472
21.67749110474979 -134.42339712412883
-77.4093461859141 -173.97725145511367
-64.12303138102672 -33.36481766103054
48.797562658561674 -139.25361493337317
79.12911394925766 139.76145519210684
-0.34102564147337944 -76.7604247302082
52.95931313798823 -178.2121908170473
49.5764281537318 -169.60054788950725
-59.62405225336765 61.937286016234935
81.4326571827834 -178.04330532777053
-2.5598098051467417 -124.94411397099762
23.021512929779092 -75.2688005499467
-0.6810496218089099 112.90823123811316
82.19859340579958 132.4638616369562
3.135996571787473 42.61380188429814
-4.320703094149323 -171.3495524870282
48.20383731157952 116.6841559921549
3.923144594391701 101.12430876788977
1.7061727027892744 172.5413558554386
14.784470640513717 2

-78.93027287212423 -69.62886058796504
-16.59041298004881 -151.23011253625097
21.299226156535724 -15.301753568017261
33.52457925792069 -22.508377216991732
46.34037951609034 132.33034029326933
-31.97369882443489 170.30087330903814
-14.744544332767674 -93.38730233012295
75.083026569092 154.80750339349902
84.22975350815096 79.81880211054238
-82.4952529385719 -156.98929192640531
14.513757147493422 -153.67190437466422
22.61109872819486 -92.10277525234298
9.66845892226523 65.90933336337193
-44.369263561511275 -27.612421678998118
-14.567083149983432 -42.2273446696729
75.96735702725371 111.6023022230504
-52.22439981157737 -111.50868907874548
12.653292434298194 -126.50682295389322
-63.87386731975218 74.9786819114126
87.3639857082768 176.4055776975731
24.704246801072145 179.98238000915325
-78.81436385279545 58.16932536139146
51.20236760002413 72.88029541357236
-24.974475019367645 85.99339012427407
-88.64961346272779 -102.44403639468909
-77.27517131828131 -24.517298512967244


In [6]:
# 4. Get the nearest city using the citipy module
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

755

In [7]:
# Search the data for keys
url2 = "http://api.openweathermap.org/data/2.5/weather?"
query_city = cities[0]

query_url =  url2 + "appid=" + weather_api_key + "&q=" + query_city

weather_response = requests.get(query_url)

weather_json = weather_response.json()

In [8]:
weather_json.keys()

dict_keys(['coord', 'weather', 'base', 'main', 'visibility', 'wind', 'clouds', 'dt', 'sys', 'timezone', 'id', 'name', 'cod'])

In [9]:
# Find the weather description in the data
pprint.pprint(weather_json)

{'base': 'stations',
 'clouds': {'all': 53},
 'cod': 200,
 'coord': {'lat': 42.6001, 'lon': -73.9662},
 'dt': 1659976118,
 'id': 5106841,
 'main': {'feels_like': 310.41,
          'humidity': 66,
          'pressure': 1015,
          'temp': 304.48,
          'temp_max': 306.02,
          'temp_min': 302.73},
 'name': 'Albany',
 'sys': {'country': 'US',
         'id': 2037353,
         'sunrise': 1659952506,
         'sunset': 1660003678,
         'type': 2},
 'timezone': -14400,
 'visibility': 10000,
 'weather': [{'description': 'broken clouds',
              'icon': '04d',
              'id': 803,
              'main': 'Clouds'}],
 'wind': {'deg': 81, 'gust': 3.13, 'speed': 0.89}}


In [10]:
# Check code before running API call
weather_json["weather"][0]["description"]

'broken clouds'

In [11]:
# 5. Perform an API call with the OpenWeatherMap

# 6. Retrieve the following information from the API call: 
# lat and lng, max temp, % humidity, % cloudiness, wind speed, 
# weather description (e.g. clouds, fog, light rain, clear sky)

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
       
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass
# Generally, it isn't good coding practice to add the pass statement to the except block. 
# Ideally, we want to handle or catch each error as it happens and do something specific 
# (e.g., add another try block or print out the error).

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | albany
Processing Record 2 of Set 1 | kailua
Processing Record 3 of Set 1 | salalah
Processing Record 4 of Set 1 | hilo
Processing Record 5 of Set 1 | rio grande
Processing Record 6 of Set 1 | rikitea
Processing Record 7 of Set 1 | lompoc
Processing Record 8 of Set 1 | vaini
Processing Record 9 of Set 1 | mar del plata
Processing Record 10 of Set 1 | sitka
Processing Record 11 of Set 1 | nizhneyansk
City not found. Skipping...
Processing Record 12 of Set 1 | nueva loja
Processing Record 13 of Set 1 | provideniya
Processing Record 14 of Set 1 | bethel
Processing Record 15 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 16 of Set 1 | mys shmidta
City not found. Skipping...
Processing Record 17 of Set 1 | atuona
Processing Record 18 of Set 1 | clarence town
Processing Record 19 of Set 1 | sampit
Processing Record 20 of Set 1 | tiksi
Processing Record 21 of Set 1 | dinsor
City not 

Processing Record 37 of Set 4 | katsuura
Processing Record 38 of Set 4 | whitehaven
Processing Record 39 of Set 4 | alta floresta
Processing Record 40 of Set 4 | brae
Processing Record 41 of Set 4 | arawa
Processing Record 42 of Set 4 | nehe
Processing Record 43 of Set 4 | sorland
Processing Record 44 of Set 4 | tiznit
Processing Record 45 of Set 4 | richards bay
Processing Record 46 of Set 4 | tuktoyaktuk
Processing Record 47 of Set 4 | touros
Processing Record 48 of Set 4 | olinda
Processing Record 49 of Set 4 | rio gallegos
Processing Record 50 of Set 4 | half moon bay
Processing Record 1 of Set 5 | lebu
Processing Record 2 of Set 5 | achisay
City not found. Skipping...
Processing Record 3 of Set 5 | tocopilla
Processing Record 4 of Set 5 | uvat
Processing Record 5 of Set 5 | ostrovnoy
Processing Record 6 of Set 5 | malanville
Processing Record 7 of Set 5 | nemuro
Processing Record 8 of Set 5 | leh
Processing Record 9 of Set 5 | kodiak
Processing Record 10 of Set 5 | tongliao
Proces

Processing Record 22 of Set 8 | vrede
Processing Record 23 of Set 8 | xadani
City not found. Skipping...
Processing Record 24 of Set 8 | ayagoz
Processing Record 25 of Set 8 | canutama
Processing Record 26 of Set 8 | kruisfontein
Processing Record 27 of Set 8 | faanui
Processing Record 28 of Set 8 | northam
Processing Record 29 of Set 8 | pervomayskoye
Processing Record 30 of Set 8 | nouakchott
Processing Record 31 of Set 8 | geraldton
Processing Record 32 of Set 8 | mandalgovi
Processing Record 33 of Set 8 | sorvag
City not found. Skipping...
Processing Record 34 of Set 8 | teeli
Processing Record 35 of Set 8 | evensk
Processing Record 36 of Set 8 | fukue
Processing Record 37 of Set 8 | egvekinot
Processing Record 38 of Set 8 | sheboygan
Processing Record 39 of Set 8 | kuito
Processing Record 40 of Set 8 | kuytun
Processing Record 41 of Set 8 | kyra
Processing Record 42 of Set 8 | xingcheng
Processing Record 43 of Set 8 | peniche
Processing Record 44 of Set 8 | mayya
Processing Record

Processing Record 13 of Set 12 | falealupo
City not found. Skipping...
Processing Record 14 of Set 12 | ndele
Processing Record 15 of Set 12 | sivas
Processing Record 16 of Set 12 | enniscorthy
Processing Record 17 of Set 12 | lumsden
Processing Record 18 of Set 12 | coria
Processing Record 19 of Set 12 | svetlaya
Processing Record 20 of Set 12 | ipswich
Processing Record 21 of Set 12 | zhezkazgan
Processing Record 22 of Set 12 | polunochnoye
Processing Record 23 of Set 12 | pushkinskiye gory
Processing Record 24 of Set 12 | tigre
Processing Record 25 of Set 12 | marcovia
Processing Record 26 of Set 12 | christchurch
Processing Record 27 of Set 12 | stornoway
Processing Record 28 of Set 12 | clarksburg
Processing Record 29 of Set 12 | springbok
Processing Record 30 of Set 12 | bajo baudo
City not found. Skipping...
Processing Record 31 of Set 12 | ngukurr
City not found. Skipping...
Processing Record 32 of Set 12 | ust-nera
Processing Record 33 of Set 12 | rungata
City not found. Skipp

Processing Record 47 of Set 15 | fethiye
Processing Record 48 of Set 15 | port-gentil
Processing Record 49 of Set 15 | jiuquan
Processing Record 50 of Set 15 | fare
Processing Record 1 of Set 16 | baoqing
Processing Record 2 of Set 16 | srednekolymsk
Processing Record 3 of Set 16 | celestun
Processing Record 4 of Set 16 | cacule
Processing Record 5 of Set 16 | ereymentau
-----------------------------
Data Retrieval Complete      
-----------------------------


In [12]:
# 7.Add the data to a new DataFrame
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Albany,42.6001,-73.9662,92.17,64,59,5.53,US,broken clouds
1,Kailua,21.4022,-157.7394,77.14,76,0,8.05,US,clear sky
2,Salalah,17.0151,54.0924,77.09,94,100,4.61,OM,drizzle
3,Hilo,19.7297,-155.0900,76.28,94,100,11.50,US,light rain
4,Rio Grande,-32.0350,-52.0986,62.71,74,74,13.82,BR,broken clouds
5,Rikitea,-23.1203,-134.9692,72.46,80,100,28.57,PF,light rain
6,Lompoc,34.6391,-120.4579,74.84,88,100,9.22,US,overcast clouds
7,Vaini,-21.2000,-175.2000,75.36,94,20,6.91,TO,few clouds
8,Mar Del Plata,-38.0023,-57.5575,53.62,79,40,5.99,AR,scattered clouds
9,Sitka,57.0531,-135.3300,55.29,88,75,0.00,US,light rain


In [13]:
# Change the column order
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]

city_data_df = city_data_df[new_column_order]

city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Albany,US,42.6001,-73.9662,92.17,64,59,5.53,broken clouds
1,Kailua,US,21.4022,-157.7394,77.14,76,0,8.05,clear sky
2,Salalah,OM,17.0151,54.0924,77.09,94,100,4.61,drizzle
3,Hilo,US,19.7297,-155.0900,76.28,94,100,11.50,light rain
4,Rio Grande,BR,-32.0350,-52.0986,62.71,74,74,13.82,broken clouds
...,...,...,...,...,...,...,...,...,...
683,Baoqing,CN,46.3317,132.2111,59.14,68,2,9.84,clear sky
684,Srednekolymsk,RU,67.4500,153.6833,57.11,65,22,6.49,few clouds
685,Celestun,MX,20.8667,-90.4000,86.20,61,92,4.29,overcast clouds
686,Cacule,BR,-14.5033,-42.2222,90.05,23,6,7.94,clear sky


In [14]:
# 8. Export the DataFrame as a CSV file, "WeatherPy_Database.csv" in the Weather_Database folder
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")